In [124]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
import finplot as fplt

import polygon_backfill as pb
# import market_cluster as mc
# import s3_datasets as s3d
# import filters as ft
import bar_samples as bs
import bar_labels as bl

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')

# .plot_bokeh(sizing_mode="scale_height")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
symbol='GLD'
# symbol = 'market'
date = '2020-10-05'
tick_type='trades'
# tick_type = None
result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

In [121]:
df = pb.backfill_dates(symbol, start_date='2020-10-01', end_date='2020-10-10', tick_type=tick_type, result_path=result_path, upload_to_s3=True)

requested 7 dates


  0%|          | 0/6 [00:00<?, ?it/s]

6 remaining dates
fetching: 2020-10-01
Downloaded: 50000 GLD ticks
Downloaded: 15591 GLD ticks
dropped 234 ticks ( 0.36 %)


 17%|█▋        | 1/6 [00:15<01:17, 15.43s/it]

/Users/bobcolner/QuantClarity/pandas-polygon/data/trades/symbol=GLD/date=2020-10-01/data.feather
fetching: 2020-10-02
Downloaded: 50000 GLD ticks
Downloaded: 4908 GLD ticks
6 duplicated trade removed
dropped 406 ticks ( 0.74 %)


...

/Users/bobcolner/QuantClarity/pandas-polygon/data/trades/symbol=GLD/date=2020-10-07/data.feather
fetching: 2020-10-08
Downloaded: 47226 GLD ticks
dropped 202 ticks ( 0.43 %)


 83%|████████▎ | 5/6 [00:58<00:11, 11.62s/it]

/Users/bobcolner/QuantClarity/pandas-polygon/data/trades/symbol=GLD/date=2020-10-08/data.feather
fetching: 2020-10-09
Downloaded: 50000 GLD ticks
Downloaded: 33225 GLD ticks
2 duplicated trade removed
dropped 264 ticks ( 0.32 %)


100%|██████████| 6/6 [01:14<00:00, 12.45s/it]

/Users/bobcolner/QuantClarity/pandas-polygon/data/trades/symbol=GLD/date=2020-10-09/data.feather


In [ ]:
# sample ticks -> bars
renko_size = 0.2
# renko_reversal = 2
thresh = {
    # GT thresholds
    'renko_size': renko_size,
    'renko_reveral_multiple': 2,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 15,
    'min_tick_count': 30,
}
# ticks_df['epoch'] = ticks_df.date_time.astype(int)
tdf = ticks_df.loc[(ticks_df.date_time.dt.hour >= 13) & (ticks_df.date_time.dt.hour <= 22)]
bars, _ = bs.build_bars(ticks_df=tdf, thresh=thresh)
len(bars)

bars_df = pd.DataFrame(bars).set_index('close_at')
# bars_df = pd.DataFrame(bars)

In [15]:
# range / price = HFT profit potential
(df.price.max() - df.price.min()) / df.price.mean()

0.014140147

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

lbars_unq = bl.get_concurrent_stats(lbars_df)
print(lbars_unq['grand_avg_unq'])

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])

In [ ]:
label_prices, price_end_at = bl.get_label_ticks(tdf, bars[31]['close_at'], 30)